In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data/sentiments_rule.csv')

In [3]:
colLst = df.columns.tolist()
colmap = {col : col.strip() for col in colLst}

In [4]:
df.rename(columns = colmap, inplace = True)

In [6]:
df = df.drop(['Unnamed: 0'],axis=1)
df = df.dropna()

In [13]:
df.columns

Index(['data', 'swn', 'vader neu', 'vader pos', 'vader neg', 'vader com',
       'bert score', 'bert label', 'roberta_val', 'rating'],
      dtype='object')

In [32]:
def Label_Sentiment(row):
       
        rating = 0
        
        vader_pos = row['vader pos']
        vader_neg = row['vader neg']
        vader_neu = row['vader neu']
        vader_com = row['vader com']
        bert_score = row['bert score']
        bert_label = row['bert label']    
        swn_score = row['swn']
        roberta_value = eval(row['roberta_val'])
        
        try:
            
            roberta_prediction = roberta_value[0]
            roberta_value = [list(i) for i in roberta_value ]        
            roberta_value = [i for i in roberta_value if i[1]=="Neutral"][0][0]
        
            if bert_label == 'POSITIVE' and roberta_value < 0.60:
                rating = 3
            elif bert_label=="POSITIVE" and roberta_value >= 0.60:
                rating = 2
            else:
                roberta_prediction =list(roberta_prediction)
                if roberta_prediction[1]=="Negative":
                    if bert_label == 'NEGATIVE' and bert_score >= 0.99 and vader_neg >= 0.3 and swn_score < -0.625:
                        rating = 1
                    else:
                        rating = 2            
                elif roberta_prediction[1]=="Neutral":
                    rating=2
                else:
                    rating=3
        except :            
            rating = 0

        return rating 

In [33]:
df['rating'] = df.apply(lambda x : Label_Sentiment(x),axis=1)

In [35]:
df.to_csv('data/sentiments_rule.csv')